
# 🧠 Alzheimer MRI Sınıflandırması: Baseline CNN Eğitimi

Bu notebook, Alzheimer MRI görüntülerini sınıflandırmak için **baseline CNN modeli**nin eğitim sürecini içermektedir.  
Amaç, transfer öğrenme ve ileri optimizasyon stratejileri öncesinde, basit bir CNN’in performansını gözlemlemek ve sınırlılıklarını ortaya koymaktır.

- Model: Custom CNN (`src/model.py`)
- Loss: CrossEntropy (class weights ile)
- Sampler: Kullanılmadı
- Amaç: Karşılaştırma için baseline performans


In [ ]:
import torch, sys, os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from prepare_data import prepare_data

# Veriyi hazırla (train/val/test split)
prepare_data()

from src.train import train_model


In [ ]:
model, history, classes = train_model(
    model_name="cnn",
    epochs=50,
    batch_size=32,
    lr=1e-3,
    use_focal=False,     # baseline: focal loss kapalı
    use_sampler=False    # baseline: sampler kapalı
)


## 📊 Eğitim Eğrileri

Aşağıda training ve validation loss/accuracy eğrileri görülmektedir.


In [ ]:
import torch, sys, os


from src.visualization import plot_training

plot_training(history, save_path="../outputs/figures/training_curves_cnn.png")



## 🔎 Confusion Matrix
Modelin sınıf bazında tahmin performansını incelemek için confusion matrix:


In [ ]:


from src.evaluate import save_confusion_matrix

# Confusion matrix training sırasında zaten kaydedildi.
# Tekrar üretmek için evaluate fonksiyonunu çağırabilirsin.



## 📑 Classification Report

Precision, recall ve F1-score değerleri:


In [ ]:

from src.evaluate import save_classification_report

# Classification report training sırasında zaten kaydedildi.
# İstersen dosyadan okuyup yazdırabilirsin.



## ✅ Sonuç ve Değerlendirme

- Baseline CNN modeli, özellikle **ModerateDemented** ve **VeryMildDemented** sınıflarında düşük performans göstermektedir.
- Genel doğruluk yaklaşık %50–60 arasında kalmaktadır.
- Bu durum, **transfer öğrenme** gibi daha güçlü stratejilere ihtiyaç olduğunu göstermektedir.


### Eğitim ve Doğrulama Eğrileri

Modelin eğitim ve doğrulama süreçleri incelendiğinde, temel bir CNN'nin öğrenme dinamikleri net bir şekilde görülmektedir.

![Training and Validation Loss and Accuracy](../images/training_curvesv2.png)

#### **Analiz:**
* **Eğitim ve Doğrulama Kaybı (`Loss`):** Eğitim kaybı istikrarlı bir şekilde azalırken, doğrulama kaybı yüksek seviyelerde dalgalanmalar gösteriyor. Bu durum, modelin eğitim verisini ezberleme eğiliminde olduğunu ve yeni verilere genelleme yapmakta zorlandığını gösterir.
* **Eğitim ve Doğrulama Doğruluğu (`Accuracy`):** Doğrulama doğruluğu, en yüksek **%58** seviyesine ulaşsa da, istikrarsızdır ve dalgalanmalar gösterir. Eğitim doğruluğu ise daha yavaş yükselir, bu da modelin öğrenme kapasitesinin sınırlı olduğunu işaret eder. Bu grafikler, modelin aşırı öğrenme (`overfitting`) problemi yaşadığını açıkça göstermektedir.

### Sınıflandırma Metrikleri

Temel modelin doğrulama seti üzerindeki performansını daha yakından incelemek, sınıflar arası denge sorununu ortaya koyar.

![Confusion Matrix](../images/confusion_matrixv2.png)

#### **Analiz:**
* **Genel Doğruluk:** %52 gibi düşük bir doğruluk değeri, temel modelin Alzheimer evrelerini güvenilir bir şekilde sınıflandıramadığını gösterir.
* **`ModerateDemented` Sınıfı:** Karışıklık matrisinde görüldüğü gibi, en nadir sınıf olan `ModerateDemented` için **hiçbir doğru tahmin yapılamamıştır**. Model bu sınıfı tamamen göz ardı etmiştir.
* **`VeryMildDemented` Sınıfı:** 359 örneğin sadece 92'si doğru tahmin edilmiştir. Geri kalan 267 örnek `MildDemented` olarak yanlış sınıflandırılmıştır.
* **Sınıf Dengesi Sorunu:** Model, `MildDemented` sınıfına 144 adet, `VeryMildDemented` sınıfına 92 adet, `NonDemented` sınıfına 339 adet yanlış tahmin yapmıştır. Bu, modelin veri setindeki dengesizliği yönetemediğini ve tahminlerini çoğunluk sınıflara kaydırdığını göstermektedir.

### Grad-CAM

![Grad-CAM Heatmap](../images/gradcamv2.png)